# Hadoop Automation Menu
### with Face Recognition using OpenCV and HAAR Cascade classifier and Ansible Automation.

#### Prepared by: Anand Soni (CSE), Neha Gupta (ECE) and, Vivek Nigam (Mathematics & Computing)

The menu is unlocked by face recognition of all the three members in the given order. After the system is unlocked we are given a menu to start **HDFS Cluster**, **Map-Reduce** Cluster and also to set NameNode, DataNode, Job Tracker and, Task Tracker. The menu can also reset the multi-node cluster. <br>
This is the client site, the code is run by the server on the server site, which is connected to this machine using **Socket Programming**. <br><br>
The Server works on **Red Hat Linux Enterprise 7.5**, and uses **Red Hat Ansible Automation** to set up all the nodes and start the services by reading some **Ansible Playbooks**.<br><br>
**Artificailly Intelligent** Face Recognition works on **HAAR Cascade Classifier** based upon the **LBPH Algorithm**, the webcam is accesed using **OpenCV** modules.

### Find My Face : OpenCV and HAAR Cascade Classifier
Training the model for all the faces which we need. **Here the model is already trained so we don't need to execute this cell.**

In [ ]:
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

cap = cv2.VideoCapture(0)
count = 0

while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        file_name_path = 'E://faces3//user' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 200: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

### Initializing the Speech Recognizer

Using the **Google© Speech API**

In [1]:
def speak(x):
    import speech_recognition as sr
    import pyttsx3
    import pyaudio
    engine=pyttsx3.init()
    engine.say(x)
    engine.runAndWait()

In [2]:
def recog():
    import speech_recognition as sr
    import pyttsx3
    import pyaudio
    mic=sr.Microphone()
    rec=sr.Recognizer()
    with mic as source:
        print("say:")
        audio=rec.listen(source)
        text=rec.recognize_google(audio)
        return(text)

### Connecting to socket

In [3]:
def send(cmd):
    import subprocess as sp
    import socket
    s=socket.socket()
    s.connect(("192.168.43.112",1235))
    s.send(cmd)

### The Authentication Process

In [4]:
def login():
    import time
    b=[]
    ii=1
    while ii <=3:
        import cv2
        import numpy as np
        from os import listdir
        from os.path import isfile, join
        print(cv2.__version__)

        data_path = 'E://Project//faces{}//'.format(ii)

        onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
        Training_Data, Labels = [], []

        for i, files in enumerate(onlyfiles):
            image_path = data_path + onlyfiles[i]
            images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            Training_Data.append(np.asarray(images, dtype=np.uint8))
            Labels.append(i)
      
        Labels = np.asarray(Labels, dtype=np.int32)
        model=cv2.face_LBPHFaceRecognizer.create()
        model.train(np.asarray(Training_Data), np.asarray(Labels))

        if(ii==3):
            print('Vivek please come in front of webcam for unlocking process')
            speak('Vivek please come in front of webcam for Unlocking process')

        if(ii==2):
            print('Neha please come in front of webcam for unlocking process')
            speak('Neha please come in front of webcam for Unlocking process')
        if(ii==1):
            print('Anand please come in front of webcam for face recognition')
            speak('Anand please come in front of webcam for Unlocking process')
        time.sleep(2)

        import cv2
        import numpy as np
        import webbrowser


        face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

        def face_detector(img, size=0.5):

            # Convert image to grayscale
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)
            if faces is ():
                return img, []

            for (x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
                roi = img[y:y+h, x:x+w]
                roi = cv2.resize(roi, (200, 200))
            return img, roi


        # Open Webcam
        cap = cv2.VideoCapture(0)

        while True:

            ret, frame = cap.read()

            image, face = face_detector(frame)

            try:
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

                # Pass face to prediction model
                # "results" comprises of a tuple containing the label and the confidence value
                results = model.predict(face)
                print(results)
                if results[1] < 500:
                    confidence = int( 100 * (1 - (results[1])/400) )
                    display_string = str(confidence) + '% Confident it is User'

                cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)

                if confidence > 75:
                    cv2.imshow('Face Recognition', image )
                    #webbrowser.open('')
                    b.append('y')
                    break
                else:
                    cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
                    cv2.imshow('Face Recognition', image )

            except:
                cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
                cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
                cv2.imshow('Face Recognition', image )
                pass

            if cv2.waitKey(1) == 13: #13 is the Enter Key
                break
        ii=ii+1
        cap.release()
        cv2.destroyAllWindows()

    print('Synergy Found')
    speak("you are successfully login in your tool")
    return(1)


### Ping me up : To check if all the Systems are connected

In [5]:
def ping(a):
    import subprocess
    b=[]
    j=0
    for i in a:
        ping = subprocess.getstatusoutput('ping {} -n 2 '.format(a[j][0]))
        if ping[0] == 0:
            b.append(i)
        else :
            print("{} ip is not reachable . Do you still want to add this y/n".format(a[j][0]))
            speak("{} ip is not reachable . Do you still want to add this y/n".format(a[j][0]))
            kuch=input('')
            if kuch == 'y':
                b.append(i)
            else:
                pass
        j=j+1
    return(b)

# The Hadoop Menu

Password : **hello**

In [ ]:
import subprocess as sp
import socket
s=socket.socket()
import pickle
import getpass
import time
print("                    Welcome to Crontab                     ")
speak('welcome to crontab')
print("You can either login by password, or by your face image")
speak("you can either login by password , or by your face image")
print("Tell me how you want to login")
speak("tell me how you want to login")
passwd='hello'
a=input()
print(a)
print()
i=3
check=0
while i>0:
    i=i-1
    if 'password' in a:
        speak("enter your password")
        in_pass=getpass.getpass("Enter your Password")
        if in_pass==passwd:
            check=1
            speak('Hello anand how may i help you')

            break
        else:
            if i==0 : 
                print("Invalid!")
                speak("You are not allowed to login")
                exit()
            if i!=0 :
                print("Please try again , you have only {} chance left".format(i))
                speak("please try again , you have only {} chance left".format(i))
    elif 'face' in a :
        b=login()
        i=0
        check=1
        break
    else:
        print('Pardon Please')
        speak('pardon please')
        a=recog()

if check==1:
    print("Here is the list of task system can perform")
    print("""
    1. Start HADOOP HDFS Cluster.
    2. Start HADOOP M-R Cluster.
    3. Setup NameNode (Master).
    4. Setup Data Nodes.
    5. Setup Job Tracker. 
    6. Setup Task Trackers.
    7. RESET HADOOP CLUSTERS.
    8. EXIT.
    """)
    namenodeon=0
    jton=0
    
    speak('choose what you want to perform out of following')
    while True:
        print('Choose what you want to do?')
        what_to_do=input()
        print(what_to_do)
        ips=[]
        
        if '1' in what_to_do or 'one' in what_to_do or 'hdfs' in what_to_do:
            if namenodeon==1 :
                send('1'.encode())
            else :
                print("First setup Hadoop MasterNode!")
            
            
        if '2' in what_to_do or 'two' in what_to_do or 'map' in what_to_do or 'reduce' in what_to_do:
            if jton==1 :
                
                send('2'.encode())
            else :
                print("First setup Hadoop MapReduce JobTracker")
            
        
        
        if '3' in what_to_do or 'three' in what_to_do or 'third' in what_to_do or 'master' in what_to_do or 'name' in what_to_do:
            extra=[]
            print('Please provide us with ip and password of system you want to add as Master ')
            speak('please provide us with ip and password of system you want to add as master ')
            print('Enter your ip:')
            ip_firsttime=input('')
            ip_password=getpass.getpass("Enter your Password")
            extra.append(ip_firsttime)
            extra.append(ip_password)
            ips.append(extra)
            final_ips=ping(ips)
            namenodeon=1
            print(final_ips)
            send(b'3')
            
            send(pickle.dumps(final_ips))
        
        
        if '4' in what_to_do or 'four' in what_to_do or 'data' in what_to_do or 'slave' in what_to_do:
            extra=[]
            print('Please provide us with ip and password of system you want to add as Slave ')
            speak('please provide us with ip and password of system you want to add as slave ')
            print('Enter your ip:')
            while True:
                print("Enter ip and password of slave you want to add")
                ip_firsttime=input('')
                ip_password=getpass.getpass("Enter your Password")
                extra.append(ip_firsttime)
                extra.append(ip_password)
                ips.append(extra)
                extra=[]
                print("Do you want to add more slave y/n")
                speak("Do you want to add more slave")
                add_more_slave=input('')
                if add_more_slave=='y':
                    pass
                elif add_more_slave=='n':
                    break
                
            final_ips=ping(ips)
            print(final_ips)
            
            send(b'4')
        
            send(pickle.dumps(final_ips))
            
        if '5' in what_to_do or 'five' in what_to_do or 'job' in what_to_do:
            extra=[]
            print('Please provide us with ip and password of system you want to add as job tracker ')
            speak('please provide us with ip and password of system you want to add as job tracker ')
            print('Enter your ip:')
            ip_firsttime=input('')
            ip_password=getpass.getpass("Enter your Password")
            extra.append(ip_firsttime)
            extra.append(ip_password)
            ips.append(extra)
            jton=1
            final_ips=ping(ips)
            print(final_ips)
            
            send(b'5')
            time.sleep(0.5)
            send(pickle.dumps(final_ips))
        
        if '6' in what_to_do or 'six' in what_to_do or 'task' in what_to_do:
            extra=[]
            print('Please provide us with ip and password of system you want to add as task tracker ')
            speak('please provide us with ip and password of system you want to add as task tracker ')
            print('Enter your ip:')
            while True:
                print("enter ip and password of task tracker you want to add")
                ip_firsttime=input('')
                ip_password=getpass.getpass("enter your password")
                extra.append(ip_firsttime)
                extra.append(ip_password)
                ips.append(extra)
                extra=[]
                print("Do you want to add more task tracker y/n")
                speak("Do you want to add more task tracker")
                add_more_slave=input('')
                if add_more_slave=='y':
                    pass
                elif add_more_slave=='n':
                    break
                
            final_ips=ping(ips)
            print(final_ips)
            send(b'6')
            send(pickle.dumps(final_ips))
            
            
        if '7' in what_to_do or 'seven' in what_to_do or 'reset' in what_to_do:
            send(b'7')    
            
        if '8' in what_to_do or 'eight' in what_to_do or 'exit' in what_to_do:
            break

                    Welcome to Crontab                     
You can either login by password, or by your face image
Tell me how you want to login
face
face

3.4.1
Anand please come in front of webcam for face recognition
(55, 53.26446760762097)
3.4.1
Neha please come in front of webcam for unlocking process
(153, 50.06561828288728)
3.4.1
Vivek please come in front of webcam for unlocking process
(145, 61.68353513216085)
Synergy Found
Here is the list of task system can perform

    1. Start HADOOP HDFS Cluster.
    2. Start HADOOP M-R Cluster.
    3. Setup NameNode (Master).
    4. Setup Data Nodes.
    5. Setup Job Tracker. 
    6. Setup Task Trackers.
    7. RESET HADOOP CLUSTERS.
    8. EXIT.
    
Choose what you want to do?
1
1
First setup Hadoop MasterNode!
Choose what you want to do?
3
3
Please provide us with ip and password of system you want to add as Master 
Enter your ip:
192.168.43.242
Enter your Password········
[['192.168.43.242', 'toor']]
Choose what you want to do?
2
2
